In [24]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump
pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows',100)
INPUT_DIR = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales'
path_pickle_dump = f'{INPUT_DIR}/'

In [25]:
# read Square csv and pickle df
df1 = pd.read_csv(f'{INPUT_DIR}/items-2021-02-01-2021-10-02.csv')
df2 = pd.read_csv(f'{INPUT_DIR}/items-2021-09-06-2022-09-07.csv')
df = pd.concat([df1, df2])
df = df.drop_duplicates()  # drop overlap between the 2 csv files
df = df.sort_values(by=['Date', 'Time'])
df = df.reset_index(drop=True)  # create new index
pickle_dump(df, path_pickle_dump, 'df_sq_download')
print(f'df.shape: {df.shape}')

df.shape: (18366, 27)


In [26]:
# Los Angeles temperature and precipitation data
# https://www.ncei.noaa.gov/cdo-web/datatools/findstation
dfw = pd.read_csv(f'{INPUT_DIR}/la_weather_2021-01-01_2022-09-28.csv')
dfw = dfw.set_index("DATE")
dfw.index = pd.to_datetime(dfw.index)  # change index to datetime
dfw = dfw.reset_index()  # index in no longer the Date column
dfw['DATE'] = dfw['DATE'].dt.strftime('%Y-%m-%d')  # change Date from datetime to str
dfw.rename(columns={'DATE':'Date'}, inplace=True)  # rename DATE to Date for dataframe merge
dfw

,Date,STATION,NAME,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08,DOW
0,2021-01-01,USW00093134,LOS ANGE...,0.22,NaN,0.0,NaN,70,45,250.0,260.0,6.0,11.0,NaN,NaN,1.0,Fri
1,2021-01-02,USW00093134,LOS ANGE...,0.45,NaN,0.0,NaN,66,47,270.0,270.0,6.9,10.1,1.0,NaN,1.0,Sat
2,2021-01-03,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,64,50,270.0,260.0,6.9,12.1,1.0,NaN,1.0,Sun
3,2021-01-04,USW00093134,LOS ANGE...,0.89,NaN,0.0,NaN,62,48,260.0,270.0,8.9,14.1,1.0,NaN,1.0,Mon
4,2021-01-05,USW00093134,LOS ANGE...,0.67,NaN,0.0,NaN,63,48,260.0,270.0,8.1,11.0,1.0,1.0,1.0,Tue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,2022-09-24,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,86,66,260.0,250.0,8.9,15.0,NaN,NaN,1.0,Sat
632,2022-09-25,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,86,66,260.0,260.0,8.9,14.1,NaN,NaN,NaN,Sun
633,2022-09-26,USW00093134,LOS ANGE...,1.12,NaN,0.0,NaN,86,66,270.0,260.0,8.9,14.1,NaN,NaN,NaN,Mon
634,2022-09-27,USW00093134,LOS ANGE...,1.34,1338.0,0.0,NaN,88,66,260.0,260.0,8.1,12.1,1.0,NaN,NaN,Tue


In [27]:
df_merged = df.merge(dfw, how='left', on='Date')
df_merged = df_merged.sort_values(by=['Date', 'Time'])
df_merged.reset_index(drop=True, inplace=True)  # drop current index
pickle_dump(df_merged, path_pickle_dump, 'df_item_sale_n_weather_raw')

In [28]:
df = pickle_load(path_pickle_dump, 'df_item_sale_n_weather_raw')
df

,Date,Time,Time Zone,Category,Item,Qty,Price Point Name,SKU,Modifiers Applied,...,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08,DOW
0,2021-07-08,17:31:07,Pacific ...,Food,choc crx,1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
1,2021-07-08,17:31:29,Pacific ...,Food,Jarred p...,1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
2,2021-07-08,17:31:47,Pacific ...,Beverage,sparkling,1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
3,2021-07-08,17:32:35,Pacific ...,Beverage,sparkling,-1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
4,2021-07-08,17:32:54,Pacific ...,Food,Jarred p...,-1.0,Regular,NaN,NaN,...,68,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Thu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18361,2022-09-04,11:21:25,Pacific ...,Food,Cinnamon...,1.0,Regular,NaN,NaN,...,77,260.0,260.0,8.1,13.0,NaN,NaN,NaN,Sun
18362,2022-09-04,11:32:24,Pacific ...,Food,Cinnamon...,1.0,Regular,NaN,NaN,...,77,260.0,260.0,8.1,13.0,NaN,NaN,NaN,Sun
18363,2022-09-04,11:39:13,Pacific ...,Food,Danish,2.0,Regular,NaN,NaN,...,77,260.0,260.0,8.1,13.0,NaN,NaN,NaN,Sun
18364,2022-09-06,09:17:15,Pacific ...,None,Croissant,72.0,NaN,NaN,NaN,...,73,270.0,270.0,8.1,12.1,NaN,NaN,NaN,Tue


In [29]:
df.columns

Index(['Date', 'Time', 'Time Zone', 'Category', 'Item', 'Qty', 'Price Point Name', 'SKU', 'Modifiers Applied', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Payment ID', 'Device Name', 'Notes', 'Details', 'Event Type', 'Location', 'Dining Option', 'Customer ID', 'Customer Name', 'Customer Reference ID', 'Unit', 'Count', 'Itemization Type', 'STATION', 'NAME', 'AWND', 'PGTM', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT08', 'DOW'], dtype='object')

In [30]:
ls = df.Date.to_list()
print(f'Date[0]: {ls[0]:>10} {type(ls[0])},   Date[-1]: {ls[-1]:>10} {type(ls[-1])}')
ls = df.Time.to_list()
print(f'Time[0]: {ls[0]:>10} {type(ls[0])},   Time[-1]: {ls[-1]:>10} {type(ls[-1])}')

Date[0]: 2021-07-08 <class 'str'>,   Date[-1]: 2022-09-06 <class 'str'>
Time[0]:   17:31:07 <class 'str'>,   Time[-1]:   09:17:15 <class 'str'>
